<a href="https://colab.research.google.com/github/jgibbons94/cse480-notebooks/blob/master/11_2_Ponder_and_Prove_NP_Completeness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ponder and Prove NP-Completeness
## Due: Saturday, 20 March 2021, 11:59 pm

<font color=red>
Good work, Jesse. Please see my comments on your answer to 16.7.2.9 and your answer to the non-book exercise.
</font>

## Watch and Learn

Watch each of the following videos:

* [P vs. NP - The Biggest Unsolved Problem in Computer Science](https://www.youtube.com/watch?v=EHp4FPyajKQ)
* [What is complexity theory? (P vs. NP explained visually)](https://www.youtube.com/watch?v=u2DLlNQiPB4)
* [P vs. NP and the Computational Complexity Zoo](https://www.youtube.com/watch?v=YX40hbAHx3s&t=3s)

For each video, state two things that interested you (theoretical or practical). Just a couple of sentences that summarize each point of interest.

### P vs. NP - The Biggest Unsolved Problem in Computer Science

All P problems are NP problems. It is unknown if any NP complete problems are also P problems.

NP complete problems are the "hardest" NP problems that have not yet been shown to be P problems. If NP Complete problems are shown to be P problems, then there are easier problems that are not NP Complete and can be shown to be P.

### What is complexity theory? (P vs. NP explained visually)
NP Complete problems tend to be hard to solve but easy to check. They are hard to solve because there are exponential possible solutions to check to find a solution.

NP means nondeterministic polynomial. They can be solved in polynomial time on a nondeterministic computer that can check all possible solutions simultaneously.

### P vs. NP and the Computational Complexity Zoo
There are many classes of problems based on how much time it takes to solve them. Some problems are difficult to check.

If an np-complete problem can be solved in polynomial time, all other np-complete problems can too. This is because they are all structurally similar.

## Exercises

Do the following Chapter 16 Exercises:

### 16.7.2.1

Unsat says it is not satisfyable. I think it contains too many contradictions.

### 16.7.2.2

Begin with a structure including a list containing only the "root" node, and the root node's color in the queue. While the queue is not empty, do the following for the next structure in the queue:
1. Assert that the color in the structure is the color of the node. On failure, return false.
2. For every child node not in the list of the current structure, push the a structure including a list containing that node after the list in the current structure, and opposite of the current color.

When the queue is empty, return True.

It follows by construction of the above algorithm that checking the 2-color condition can be done with a breadth-first search.

### 16.7.2.8

In a 5-clique, every node is connected to every other node, so there are $5! = 5 * 4 * 3 * 2 * 1 = 120$ different hamiltonian cycles if we draw distinctions at where they start and 4!=24 hamiltonian cycles if we draw an equivalence at the order of nodes that can be visited in a directed cycle. For an n-clique there are n! Hamiltonian cycles.

### 16.7.2.9

We already have a proof in the book that proving some graph $\phi$ is a k-clique is NPC. This problem shows that graph G is an n/2-clique or graph G is an n/2 + 1 -clique or ... or graph G is an n - 1-clique or graph G is an n-clique. This ia a $k^{n/2} + k^{n/2 + 1} + ... + k^{n-1} + k^n \rightarrow O(k^n)$ algorithm.

In other words, this maps to the k-clique algorithm where $k=n/2$.

<font color=red>
Remember the *complete form* of an NP-Completeness proof?
</font>

There are 6 steps:

1. Describe the problem as a language.
2. Show that the language is in NP.
3. Select a known NP-complete problem XYZ.
4. Construct a transformation f from XYZ to the new (unknown) language.
5. Prove both directions of the transformation (if and only if).
6. Prove that f is a polynomial time transformation (reduction).

Step 1:

Describe the language HALF-CLIQUE as

$\{\langle G \rangle : G$ is an undirected graph having a complete subgraph with at least $m/2$ nodes, where $m$ is the number of nodes in $G\}$.

Step 2:

We need to show HALF-CLIQUE $\in$ NP. The certificate is simply the clique.

Step 3:

Choose CLIQUE as the known NP-complete language/problem.

Steps 4-6:

A polynomial time mapping reduction from CLIQUE to HALF-CLIQUE goes like this:

The input to the reduction is a pair $\langle G, k\rangle$ and the reduction produces the graph $\langle H\rangle$ as output where $H$ depends on $k$ versus $m$:

1. If $k = m/2$ then $H = G$.
2. If $k < m/2$, then $H$ is the graph obtained from $G$ by adding $j = m - 2k$ nodes, each one connected to every one of the original nodes and to each other. Thus $H$ has $m + j = 2m - 2k$ nodes, and so $G$ has a $k$-clique iff $H$ has a clique of size $k + j = m - k$ and so $\langle G, k\rangle \in$ CLIQUE iff $\langle H\rangle \in$ HALF-CLIQUE.
3. If $k > m/2$, then $H$ is the graph obtained by adding $j = 2k - m$ nodes to $G$ without adding any additional links. Thus $H$ has $m + j = 2k$ nodes, and so $G$ has a $k$-clique iff $H$ has a clique of size $k$. Therefore $\langle G, k\rangle \in$ CLIQUE iff $\langle H\rangle \in$ HALF-CLIQUE.

Compare [this poster's comments](https://cs.stackexchange.com/questions/1176/half-clique-np-complete-problem).


### 16.7.2.10

In [ ]:
!pip install pycosat

     |████████████████████████████████| 71kB 4.3MB/s 
  Created wheel for pycosat: filename=pycosat-0.6.3-cp37-cp37m-linux_x86_64.whl size=143812 sha256=8b9821dc1cfac98c95dd310df454e7ae2ceba75ffe254ec590827261ff768199
  Stored in directory: /root/.cache/pip/wheels/c4/67/ff/5570304e45814eccef48a3c69c3af25d0456ed3a34eddbbe38
Successfully built pycosat


In [ ]:
%%python
# Author: Nicholas Pilkington, 2015
# License: MIT
# Blog Post: https://nickp.svbtle.com/sudoku-satsolver

import pycosat

N = 9
M = 3

def exactly_one(variables):
    cnf = [ variables ]
    n = len(variables)

    for i in range(n):
        for j in range(i+1, n):
            v1 = variables[i]
            v2 = variables[j]
            cnf.append([-v1, -v2])

    return cnf

def transform(i, j, k):
    return i*N*N + j*N + k + 1

def inverse_transform(v):
    v, k = divmod(v-1, N)
    v, j = divmod(v, N)
    v, i = divmod(v, N)
    return i, j, k

if __name__ == '__main__':
    cnf = []

    # Cell, row and column constraints
    for i in range(N):
        for s in range(N):
            cnf = cnf + exactly_one([ transform(i, j, s) for j in range(N) ])
            cnf = cnf + exactly_one([ transform(j, i, s) for j in range(N) ])

        for j in range(N):
            cnf = cnf + exactly_one([ transform(i, j, k) for k in range(N) ])

    # Sub-matrix constraints
    for k in range(N):
        for x in range(M):
            for y in range(M):
                v = [ transform(y*M + i, x*M + j, k) for i in range(M) for j in range(M)]
                cnf = cnf + exactly_one(v)

    # See contribution from @GregoryMorse below
    cnf = { frozenset(x) for x in cnf }
    cnf = list(cnf)

    # A 16-constraint Sudoku
    constraints = [
        (0, 3, 7),
        (1, 0, 1),
        (2, 3, 4),
        (2, 4, 3),
        (2, 6, 2),
        (3, 8, 6),
        (4, 3, 5),
        (4, 5, 9),
        (5, 6, 4),
        (5, 7, 1),
        (5, 8, 8),
        (6, 4, 8),
        (6, 5, 1),
        (7, 2, 2),
        (7, 7, 5),
        (8, 1, 4),
        (8, 6, 3),

    ]

    cnf = cnf + [[transform(z[0], z[1], z[2])-1] for z in constraints]

    for solution in pycosat.itersolve(cnf):
        X = [ inverse_transform(v) for v in solution if v > 0]
        for i, cell in enumerate(sorted(X, key=lambda h: h[0] * N*N + h[1] * N)):
            print(cell[2]+1, end=" ")
            if (i+1) % N == 0: print("")

2 6 4 7 1 5 8 3 9 
1 3 7 8 9 2 6 4 5 
5 9 8 4 3 6 2 7 1 
4 2 3 1 7 8 5 9 6 
8 1 6 5 4 9 7 2 3 
7 5 9 6 2 3 4 1 8 
3 7 5 2 8 1 9 6 4 
9 8 2 3 6 4 1 5 7 
6 4 1 9 5 7 3 8 2 


In [ ]:

%%python
# Author: Nicholas Pilkington, 2015
# License: MIT
# Blog Post: https://nickp.svbtle.com/sudoku-satsolver

import pycosat

N = 9
M = 3

def exactly_one(variables):
    cnf = [ variables ]
    n = len(variables)

    for i in range(n):
        for j in range(i+1, n):
            v1 = variables[i]
            v2 = variables[j]
            cnf.append([-v1, -v2])

    return cnf

def transform(i, j, k):
    return i*N*N + j*N + k + 1

def inverse_transform(v):
    v, k = divmod(v-1, N)
    v, j = divmod(v, N)
    v, i = divmod(v, N)
    return i, j, k

if __name__ == '__main__':
    cnf = []

    # Cell, row and column constraints
    for i in range(N):
        for s in range(N):
            cnf = cnf + exactly_one([ transform(i, j, s) for j in range(N) ])
            cnf = cnf + exactly_one([ transform(j, i, s) for j in range(N) ])

        for j in range(N):
            cnf = cnf + exactly_one([ transform(i, j, k) for k in range(N) ])

    # Sub-matrix constraints
    for k in range(N):
        for x in range(M):
            for y in range(M):
                v = [ transform(y*M + i, x*M + j, k) for i in range(M) for j in range(M)]
                cnf = cnf + exactly_one(v)

    # See contribution from @GregoryMorse below
    cnf = { frozenset(x) for x in cnf }
    cnf = list(cnf)

    # A 16-constraint Sudoku
    constraints = [
        (0, 0, 2),
        (1, 0, 1),
        (2, 3, 4),
        (2, 4, 3),
        (2, 6, 2),
        (3, 8, 6),
        (4, 3, 5),
        (4, 5, 9),
        (5, 6, 4),
        (5, 7, 1),
        (5, 8, 8),
        (6, 4, 8),
        (6, 5, 1),
        (7, 2, 2),
        (7, 7, 5),
        (8, 1, 4),
        (8, 6, 3),

    ]

    cnf = cnf + [[transform(z[0], z[1], z[2])-1] for z in constraints]

    for solution in pycosat.itersolve(cnf):
        X = [ inverse_transform(v) for v in solution if v > 0]
        for i, cell in enumerate(sorted(X, key=lambda h: h[0] * N*N + h[1] * N)):
            print(cell[2]+1, end=" ")
            if (i+1) % N == 0: print("")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




### 16.7.2.12

Consider:
$$\Phi = (x + y + y) \cdot{} (x + !y + !y) \cdot{} (!x + y + y) \cdot (!x + !y + !y)$$

For any assignment, we have one clause whose literals are all true and one clause whose literals are all false. If we leave out any one clause, that is not the case, as $\Phi$ becomes satisfiable.

I'm fairly certain there isn't anything left in the proof for me to add. We have proven that if there is one clause whose literals are all false and one whose literals are all true, the 3-CNF is not satisfiable because false and true will always be false. We have also shown that if we remove one of the two clauses so there is no such pair, the equation becomes satisfiable. Therefore, the equation is unsatisfiable iff for any variable assignment there is one clause with all literals true and another clause with all literals false.

## Explore a Powerful Tool

Do the following NON-book exercise:

SAT, while being NP-Complete, is a "workhorse of a tool." This exercise asks you to get a taste of running the CryptoMinisat tool on a non-trivial SAT formula. Click on https://msoos.github.io/cryptominisat_web/ to access this tool. When it comes up, it has a prefilled formula. There is a Play button that you can click, whereupon it solves the SAT instance.

This exercise asks you to replace this SAT instance with something bigger: specifically, the Pigeonhole problem (hole6.cnf) from: https://people.sc.fsu.edu/~jburkardt/data/cnf/cnf.html

Just click the above link, get the hole6.cnf file, and plunk the CNF into the buffer.

Hit Play and report on the execution time (you can just look at your phone's clock). If under 2 seconds, say "negligible" for your answer!

How much time would such a problem take through brute-force enumeration of $2^n$ combinations on a computer that takes a microsecond per variable combination (the $n$ is the number of variables used in the Pigeonhole problem)? HINT: Here is how you read the contents of a CNF file:

```
c File:  hole6.cnf <--- these are comment lines - starts wth a "C"
c...
c
p cnf 42 133
-1     -7
-1     -13
...
0 0
<--- CRUCIAL !! Tells you there are 42 variables and 133 clauses
<--- This line says (!x1 + !x7). The "0" is just end-of-a-clause marker!
<--- This line says (!x1 + !x13)
 12     11     10     9      8      7    0 <--- This clause reads
                                          (x12 + x11 + x10 + x9 + x8 + x7)
```

Okay, now you have all the info you need to calculate the time it takes to enumerate $2^n$ combinations!!



Your (two-part) answer goes here.

1. CryptoMinisat runtime: neglegible
2. $2^n$ runtime estimation: 50 days

$2^42 * 133$ calculations
= about $2^50$ calculations.
Where $2^10 = 1024$
And $1024^5 \approx 1000^5$ (I know it's larger, but not by decimal figures.)
And the computer does about $2*1000^3$ calculations per second (closer to 3GHz actually)
And 1 hour  $\approx 3 * 1000^1$ seconds (3600 seconds in 1 hour)
And 50 days is $12 * 10^2 \approx 1000$ hours. (12 * 2 * 5 * 10 hours)
Is it sloppy? Yes. Is it accurate? I probably won't find out any time soon.


<font color=red>
The question just asked how long it would take to enumerate $2^{42}$ combinations. $2^{42}$ microseconds is roughly $4$ million seconds, or about a month.
</font>


##  Read and Learn

List six facts that you found interesting about Boolean SAT in these articles:

https://cacm.acm.org/magazines/2009/8/34498-boolean-satisfiability-from-theoretical-hardness-to-practical-success/fulltext

and

https://en.wikipedia.org/wiki/Boolean_satisfiability_problem

Anything that interested you is fine – theoretical or practical. Please write 1-2 sentences per point that interested you.

The term overconstrained refers to a problem with no solution. Such solutions have constraints that contradict each other.

Problems with nonboolean variables can be expressed with simple logic. These solutions look very clever and usually involve describing the equality or inequality of variables.

When asking about "eventually" is s useful to limit the time to a specific number of steps. That way, there is a definite answer that does not depend on a program finishing. Jove does this to avoid the halting problem.

SAT was the first known NP-complete problem. All other NP-complete problems can therefore be traced to the SAT problem at the root.

NP-completeness only concerns the worst-case of an algorithm. There are known ways of solving NP-complete problems that are usually more efficient.

Subsets of SAT are in P, but the entire SAT set is considered NP-complete. If the entire set can be partitioned into P and NP-hard or NP-complete, then it might be easier to organize a solution to the P VS NP problem.